In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
train = pd.read_csv("../data/train.csv").sample(frac=1.0)
test = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

f_cols = [col for col in train.columns if col not in ["id", "target"]]

In [3]:
def get_predictions(x):
    return [1 if xi >= 0.5 else 0 for xi in x]

In [4]:
X, y = train[f_cols].values, train["target"].values

In [5]:
X.shape

(250, 300)

In [6]:
loo = LeaveOneOut()
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(loo.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train = y[train_index]
    clf = LogisticRegression(random_state=0, C=1.).fit(X_train, y_train)
    preds[test_index] = clf.predict_proba(X_test)[:,1]

print(f"Models AUC score: {roc_auc_score(y, preds)}")
print(classification_report(y, get_predictions(preds)))

Models AUC score: 0.7685416666666666
              precision    recall  f1-score   support

         0.0       0.63      0.52      0.57        90
         1.0       0.75      0.82      0.79       160

    accuracy                           0.72       250
   macro avg       0.69      0.67      0.68       250
weighted avg       0.71      0.72      0.71       250



In [7]:
# Logreg MLE
log_reg = LogisticRegression(random_state=0, C=1.0).fit(X, y)

In [8]:
X_test = test[f_cols].values
y_pred = log_reg.predict_proba(X_test)[:,1]
submission["target"] = y_pred

In [9]:
# LB result 0.74
submission.to_csv("../submissions/09_logreg_MLE.csv", index=False)

In [10]:
# Score 0.740
submission.head()

,id,target
0,250,0.247957
1,251,0.065619
2,252,0.758937
3,253,0.999923
4,254,0.278511


# Gaussian logreg

## MAP l2 estimate C=0.3

In [11]:
loo = LeaveOneOut()
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(loo.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train = y[train_index]
    clf = LogisticRegression(random_state=0, C=.3, penalty="l2", solver='liblinear').fit(X_train, y_train)
    preds[test_index] = clf.predict_proba(X_test)[:,1]

print(f"Models AUC score: {roc_auc_score(y, preds)}")
print(classification_report(y, get_predictions(preds)))

Models AUC score: 0.7697916666666667
              precision    recall  f1-score   support

         0.0       0.58      0.62      0.60        90
         1.0       0.78      0.74      0.76       160

    accuracy                           0.70       250
   macro avg       0.68      0.68      0.68       250
weighted avg       0.71      0.70      0.70       250



In [12]:
log_reg = LogisticRegression(random_state=0, C=.3, penalty="l2").fit(X_train, y_train)

In [13]:
# 0.741
X_test = test[f_cols].values
y_pred = log_reg.predict_proba(X_test)[:,1]
submission["target"] = y_pred
submission.to_csv("../submissions/12_logreg_MAP_l2_c_05.csv", index=False)
submission.head()

,id,target
0,250,0.422276
1,251,0.147829
2,252,0.687185
3,253,0.999194
4,254,0.343320


# Slap-and-spike prior

In [16]:
import pymc3 as pm
import theano as tt
from scipy.special import expit
from scipy.stats import norm, bernoulli

In [18]:
X, y = train[f_cols].values, train["target"].values

This is our slab and spike model

$$a \sim \mathcal{N}(0, 3)$$
$$\gamma_i \sim Bernoulli(p=0.1)$$
$$\alpha_i|\sigma_\beta \sim \mathcal{N}(0, \sigma_\beta)$$
$$e \sim \mathcal{N}(0, \sigma^2_eI_n)$$
$$y \sim \frac{1}{1+exp(-(a + \sum_{i=1}^N \gamma_i \alpha_i x_i + e))}$$

The model parameters are $\theta = \{\gamma_i, \alpha_i\}_i^N $. The $\gamma_i$ and $\alpha_i$ are modelled IID.

In [73]:
prob = 0.1
a_mu = 0
a_var = 3
gamma_var = 1
with pm.Model() as model:
    # priors inclusion probability
    gamma_i = pm.Bernoulli("gamma_i", prob, shape=X.shape[1])
    # a is the interception
    a = pm.Normal("a", mu=a_mu, sd=a_var)
    # The prior for the features varibles which are included
    alpha = pm.Normal("alpha", mu=0, sd=gamma_var, shape=X.shape[1])
    # Deterministic function
    p = pm.math.dot(X,gamma_i * alpha) 
    # Likelihood
    y_obs = pm.Bernoulli("y_obs", pm.invlogit(p + a),  observed=y)
 

In [50]:
with model:
    trace = pm.sample(4000, random_seed = 4816, cores = 1, progressbar = True, chains = 1)

/var/folders/bx/2b_70g054qg3qt5jhnw30xv40000gn/T/ipykernel_46993/1538975003.py:2: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(4000, random_seed = 4816, cores = 1, progressbar = True, chains = 1)
Sequential sampling (1 chains in 1 job)
CompoundStep
>BinaryGibbsMetropolis: [gamma_i]
>NUTS: [alpha, a]


Sampling 1 chain for 1_000 tune and 4_000 draw iterations (1_000 + 4_000 draws total) took 144 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


# Map estimate

The log loss for the spike and slap prior
$$p(\theta) = p(\{\gamma_i, \alpha_i\}_i^N) = \prod_{i=1}^N p(\gamma_i) p(\alpha_i)$$

$$\log p(\theta) = \sum_{i=1}^N \log Bernoulli(\gamma_i | p=0.1) + \log \mathcal{N}(\alpha_i | \mu=0, \sigma_\beta=3)$$

In [58]:
def spike_slab_log_prior(gamma: np.array, alpha: np.array, p, gamma_mu=0, sigma_beta=3):
    return (bernoulli.logpmf(gamma, p=p) + norm.logpdf(alpha, loc=gamma_mu, scale=sigma_beta)).sum()

The negative log likelihood function, i.e. cross entropy

$$E(\mathbf{w}) = -log(p(\pmb{\mathbf{t}}|\pmb{\mathbf{x}},\mathbf{w})) \nonumber$$

$$= - \sum_{n=1}^N \left( t_n \ln y(\mathbf{x}_n) + (1-t_n) \ln (1-y(\mathbf{x}_n)) \right)$$
  

In [59]:
def log_likelihood(a, gamma, alpha, X, T):
    y_x = expit(a + np.dot(X, np.transpose(gamma*alpha)))
    return (T*np.log(y_x) + ((1-T)*np.log(1-y_x))).sum()

In [60]:
prob = 0.1
gamma_mu = 0
sigma_beta = 3

def find_spike_slab_MAP(trace, X, y, prob, gamma_mu, sigma_beta):
    min_loss = np.inf
    cur_min = -1
    for i in range(len(trace)):
        tmp_trace = trace[i]
        tmp_spike_slab_log_prior = spike_slab_log_prior(tmp_trace["gamma_i"], tmp_trace["alpha"], p=prob, gamma_mu=gamma_mu, sigma_beta=sigma_beta)
        tmp_log_likelihood = log_likelihood(tmp_trace["a"], tmp_trace["gamma_i"], tmp_trace["alpha"], X, y)
        neq_loss = -(tmp_log_likelihood + tmp_spike_slab_log_prior)
        if neq_loss <= min_loss:
            min_loss = neq_loss
            cur_min = i
    return trace[cur_min]
    

In [61]:
map_trace = find_spike_slab_MAP(trace, X, y, prob, gamma_mu, sigma_beta)

In [62]:
map_estimate = map_trace["gamma_i"] * map_trace["alpha"]

X_test = test[f_cols].values
map_preds = expit(map_trace["a"] + np.dot(X_test, np.transpose(map_estimate)))

In [63]:
map_preds

array([0.99354538, 0.96931246, 0.92504746, ..., 0.01800972, 0.97470334,
       0.60972093])

In [64]:
# 0.803
submission["target"] = map_preds
submission.to_csv("../submissions/14_sas_MAP_logreg.csv", index=False)
submission.head()

,id,target
0,250,0.993545
1,251,0.969312
2,252,0.925047
3,253,0.997561
4,254,0.092070


# Full Bayesian



In [65]:
results = pd.DataFrame({'var': np.arange(300), 
                        'inclusion_probability':np.apply_along_axis(np.mean, 0, trace['gamma_i']),
                       'alpha':np.apply_along_axis(np.mean, 0, trace['alpha']),
                       'alpha_given_inclusion': np.apply_along_axis(np.sum, 0, trace['gamma_i']*trace['alpha'])
                            /np.apply_along_axis(np.sum, 0, trace['gamma_i'])
                       })

In [66]:
results.sort_values('inclusion_probability', ascending = False).head(10)


,var,inclusion_probability,alpha,alpha_given_inclusion
33,33,1.00000,2.199171,2.199171
65,65,1.00000,1.872293,1.872293
217,217,0.99900,-1.325332,-1.326757
91,91,0.99800,-1.381903,-1.385135
199,199,0.97700,1.221466,1.249742
73,73,0.96650,-1.100357,-1.136644
108,108,0.80075,-0.772218,-0.968407
295,295,0.78525,-0.753310,-0.956966
117,117,0.69400,-0.660401,-0.940196
189,189,0.69125,-0.623186,-0.931997


## Bayesian inference

In [67]:
estimate = trace['alpha'] * trace['gamma_i'] 
X_test = test[f_cols].values
preds = np.apply_along_axis(np.mean, 1, expit(trace['a'] + np.dot(X_test, np.transpose(estimate) )) )


In [68]:
# 0.854
submission["target"] = preds
submission.to_csv("../submissions/15_sas_Bayesian_logreg.csv", index=False)
submission.head()

,id,target
0,250,0.857138
1,251,0.740193
2,252,0.861291
3,253,0.989098
4,254,0.488646


In [69]:
preds[:10]

array([0.85713836, 0.74019269, 0.86129088, 0.98909845, 0.48864633,
       0.37146803, 0.41335168, 0.22870848, 0.93852218, 0.25608885])

In [70]:
map_preds[:10]

array([0.99354538, 0.96931246, 0.92504746, 0.99756073, 0.0920696 ,
       0.61137463, 0.02696762, 0.24671615, 0.77067657, 0.02368206])

In [71]:
(map_preds[map_preds > 0.5]).mean(), (map_preds[map_preds <= 0.5]).mean()

(0.9286543358915618, 0.1197995855721606)

In [72]:
(preds[preds > 0.5]).mean(), (preds[preds <= 0.5]).mean()

(0.8285610321005993, 0.2558121580687975)